# Complex Queries with PySpark
## LinkedIn Job Postings Analysis using Spark SQL

In [ ]:
# Install PySpark if not already installed
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, desc, avg, rank, window
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
# Initialize Spark Session
spark = SparkSession.builder \
    .appName("LinkedIn Job Analysis") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

print(f"Spark Version: {spark.version}")
print(f"Spark Session Created: {spark}")

## Load Data into Spark DataFrames

In [ ]:
# Load all datasets
postings = spark.read.csv(r"../data/postings_cleaned.csv", header=True, inferSchema=True)
job_skills = spark.read.csv(r"../data/jobs/job_skills.csv", header=True, inferSchema=True)
skill_map = spark.read.csv(r"../data/mappings/skills.csv", header=True, inferSchema=True)
job_industries = spark.read.csv(r"../data/jobs/job_industries.csv", header=True, inferSchema=True)
industry_map = spark.read.csv(r"../data/mappings/industries.csv", header=True, inferSchema=True)
company_ind = spark.read.csv(r"../data/companies/company_industries.csv", header=True, inferSchema=True)

print("Data loaded successfully!")
print(f"Postings count: {postings.count()}")
print(f"Job skills count: {job_skills.count()}")
print(f"Industries count: {industry_map.count()}")

## Complex Query 1: Top Skills by Industry with Ranking
This query demonstrates:
- Multiple JOIN operations
- Window functions for ranking
- GROUP BY and aggregations

In [ ]:
# Clean industry map (drop nulls)
industry_map_clean = industry_map.dropna()

# Complex query: Top 10 skills per industry
skills_by_industry = job_skills \
    .join(skill_map, "skill_abr") \
    .join(job_industries, "job_id") \
    .join(industry_map_clean, "industry_id") \
    .groupBy("industry_name", "skill_name") \
    .agg(count("*").alias("skill_count")) \
    .withColumn("rank", rank().over(Window.partitionBy("industry_name").orderBy(desc("skill_count")))) \
    .filter(col("rank") <= 10) \
    .orderBy("industry_name", "rank")

print("=== Top 10 Skills by Industry ===")
skills_by_industry.show(50, truncate=False)

In [ ]:
# Convert to pandas for visualization
skills_by_industry_pd = skills_by_industry.toPandas()
print(f"Total rows: {len(skills_by_industry_pd)}")
skills_by_industry_pd.head(20)

## Complex Query 2: Average Skills Required by Industry
Analyzes skill complexity across different industries

In [ ]:
# Query: Average number of skills per job by industry
multi_skill_jobs = job_skills \
    .groupBy("job_id") \
    .agg(count("skill_abr").alias("num_skills")) \
    .join(job_industries, "job_id") \
    .join(industry_map_clean, "industry_id") \
    .groupBy("industry_name") \
    .agg(
        avg("num_skills").alias("avg_skills_required"),
        count("job_id").alias("total_jobs")
    ) \
    .orderBy(desc("avg_skills_required"))

print("=== Average Skills Required by Industry ===")
multi_skill_jobs.show(30, truncate=False)

In [ ]:
# Convert to pandas
avg_skills_pd = multi_skill_jobs.toPandas()
avg_skills_pd.head(20)

## Complex Query 3: Cross-Industry Skill Overlap Analysis
Identifies skills that are common across multiple industries

In [ ]:
# Query: Skills appearing in multiple industries
cross_industry_skills = job_skills \
    .join(skill_map, "skill_abr") \
    .join(job_industries, "job_id") \
    .join(industry_map_clean, "industry_id") \
    .select("skill_name", "industry_name") \
    .distinct() \
    .groupBy("skill_name") \
    .agg(count("industry_name").alias("num_industries")) \
    .orderBy(desc("num_industries"))

print("=== Skills Across Multiple Industries ===")
cross_industry_skills.show(30, truncate=False)

In [ ]:
cross_industry_pd = cross_industry_skills.toPandas()
cross_industry_pd.head(20)

## Save Results for Later Use

In [ ]:
# Create output directory
import os
os.makedirs("../analytics_output/query_results", exist_ok=True)

# Save as parquet for efficient storage
skills_by_industry.write.mode("overwrite").parquet("../analytics_output/query_results/skills_by_industry")
multi_skill_jobs.write.mode("overwrite").parquet("../analytics_output/query_results/avg_skills_by_industry")
cross_industry_skills.write.mode("overwrite").parquet("../analytics_output/query_results/cross_industry_skills")

# Also save as CSV for easy viewing
skills_by_industry_pd.to_csv("../analytics_output/query_results/skills_by_industry.csv", index=False)
avg_skills_pd.to_csv("../analytics_output/query_results/avg_skills_by_industry.csv", index=False)
cross_industry_pd.to_csv("../analytics_output/query_results/cross_industry_skills.csv", index=False)

print("✅ Results saved successfully!")

## Visualizations

In [ ]:
# Create visuals directory
os.makedirs("../analytics_output/visuals", exist_ok=True)

# Plot 1: Top 10 Skills in Top 5 Industries
top_5_industries = avg_skills_pd.nlargest(5, 'avg_skills_required')['industry_name'].tolist()

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Top 10 Skills by Industry', fontsize=16, fontweight='bold')

for idx, industry in enumerate(top_5_industries):
    data = skills_by_industry_pd[skills_by_industry_pd['industry_name'] == industry].head(10)
    ax = axes[idx // 3, idx % 3]
    sns.barplot(data=data, y='skill_name', x='skill_count', ax=ax, palette='viridis')
    ax.set_title(f"{industry}", fontsize=10, fontweight='bold')
    ax.set_xlabel("Job Postings")
    ax.set_ylabel("")

# Hide the 6th subplot
axes[1, 2].axis('off')

plt.tight_layout()
plt.savefig("../analytics_output/visuals/top_skills_by_industry.png", dpi=300, bbox_inches='tight')
plt.show()

print("✅ Visualization 1 saved!")

In [ ]:
# Plot 2: Average Skills Required by Industry
plt.figure(figsize=(12, 8))
top_20_avg = avg_skills_pd.head(20)
sns.barplot(data=top_20_avg, y='industry_name', x='avg_skills_required', palette='magma')
plt.title('Average Number of Skills Required by Industry (Top 20)', fontsize=14, fontweight='bold')
plt.xlabel('Average Skills Required', fontsize=12)
plt.ylabel('Industry', fontsize=12)
plt.tight_layout()
plt.savefig("../analytics_output/visuals/avg_skills_by_industry.png", dpi=300, bbox_inches='tight')
plt.show()

print("✅ Visualization 2 saved!")

In [ ]:
# Plot 3: Cross-Industry Skills (Most Versatile Skills)
plt.figure(figsize=(12, 8))
top_30_cross = cross_industry_pd.head(30)
sns.barplot(data=top_30_cross, y='skill_name', x='num_industries', palette='coolwarm')
plt.title('Most Versatile Skills (Appearing Across Industries)', fontsize=14, fontweight='bold')
plt.xlabel('Number of Industries', fontsize=12)
plt.ylabel('Skill Name', fontsize=12)
plt.tight_layout()
plt.savefig("../analytics_output/visuals/cross_industry_skills.png", dpi=300, bbox_inches='tight')
plt.show()

print("✅ Visualization 3 saved!")

## Summary Statistics for Demo

In [ ]:
print("="*60)
print("COMPLEX QUERIES SUMMARY")
print("="*60)
print(f"\n📊 Total Industries Analyzed: {industry_map_clean.count()}")
print(f"📊 Total Unique Skills: {skill_map.count()}")
print(f"📊 Total Job Postings: {postings.count()}")
print(f"\n🔍 Query 1: Identified top 10 skills for each industry")
print(f"🔍 Query 2: Calculated average skills required across {avg_skills_pd.shape[0]} industries")
print(f"🔍 Query 3: Found {cross_industry_pd.shape[0]} skills used across industries")
print(f"\n✅ All results saved to: ../analytics_output/query_results/")
print(f"✅ All visualizations saved to: ../analytics_output/visuals/")
print("="*60)

In [ ]:
# Stop Spark session
# spark.stop()